## Задача 1 - Анализ данных ПК

Для ПК создать таблицы-описания и оценить возможные значения ПК через 10 лет. На основе этой оценки задать типы ПК.

In [1]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Float, UniqueConstraint
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker

Base = declarative_base()

class Book(Base):
    __tablename__ = 'books'
    isbn = Column(String, primary_key=True)
    title = Column(String)
    year_published = Column(Integer)
    price = Column(Float)
    authors = relationship('Author', secondary='book_authors')
    # ... other relationships

class Author(Base):
    __tablename__ = 'authors'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    address = Column(String)
    homepage_url = Column(String)
    books = relationship('Book', secondary='book_authors')

class BookAuthor(Base):
    __tablename__ = 'book_authors'
    book_isbn = Column(String, ForeignKey('books.isbn'), primary_key=True)
    author_id = Column(Integer, ForeignKey('authors.id'), primary_key=True)

# ... другие классы для каждой сущности

# Создание инстанса движка, который будет взаимодействовать с базой данных
engine = create_engine('sqlite:///:memory:')

# Создание всех таблиц в базе данных
Base.metadata.create_all(engine)

# Создание сессии для взаимодействия с базой данных
Session = sessionmaker(bind=engine)
session = Session()

# Пример добавления данных в базу
author = Author(name='John Doe', address='1234 Fictional St', homepage_url='http://johndoe.com')
book = Book(isbn='1234567890', title='The Great Book', year_published=2021, price=29.99)
book.authors.append(author)
session.add(book)
session.commit()

# Вывод книг из базы данных
books = session.query(Book).all()
for book in books:
    print(book.title, book.isbn, [author.name for author in book.authors])


ModuleNotFoundError: No module named 'sqlalchemy'

## Задача 2 - Платный прием пациентов

Расчет зарплаты врачей на основе стоимости приема и процента отчисления.

In [ ]:
import sqlite3

In [ ]:
from datetime import datetime

In [4]:
import sqlite3
from datetime import datetime

# Подключение к базе данных SQLite
conn = sqlite3.connect(':memory:')  # Создание базы данных в памяти
cursor = conn.cursor()

# Создание таблиц
cursor.execute('''
CREATE TABLE doctors (
    id INTEGER PRIMARY KEY,
    name TEXT,
    specialty TEXT,
    fee_percentage REAL,
    consultation_fee REAL
)''')

cursor.execute('''
CREATE TABLE patients (
    id INTEGER PRIMARY KEY,
    name TEXT,
    address TEXT,
    phone TEXT
)''')

cursor.execute('''
CREATE TABLE appointments (
    id INTEGER PRIMARY KEY,
    doctor_id INTEGER,
    patient_id INTEGER,
    appointment_date TEXT,
    consultation_fee REAL,
    FOREIGN KEY(doctor_id) REFERENCES doctors(id),
    FOREIGN KEY(patient_id) REFERENCES patients(id)
)''')

cursor.execute('''
CREATE TABLE receipts (
    id INTEGER PRIMARY KEY,
    appointment_id INTEGER,
    amount_paid REAL,
    receipt_date TEXT,
    FOREIGN KEY(appointment_id) REFERENCES appointments(id)
)''')

cursor.execute('''
CREATE TABLE salaries (
    id INTEGER PRIMARY KEY,
    doctor_id INTEGER,
    appointment_id INTEGER,
    gross_salary REAL,
    income_tax REAL,
    net_salary REAL,
    FOREIGN KEY(doctor_id) REFERENCES doctors(id),
    FOREIGN KEY(appointment_id) REFERENCES appointments(id)
)''')

# Вставка данных
cursor.execute('INSERT INTO doctors (name, specialty, fee_percentage, consultation_fee) VALUES (?, ?, ?, ?)',
               ('Dr. Smith', 'Cardiologist', 0.6, 100.0))

cursor.execute('INSERT INTO patients (name, address, phone) VALUES (?, ?, ?)',
               ('John Doe', '123 Maple St', '555-1234'))

# Получение ID вставленных записей
doctor_id = cursor.lastrowid
patient_id = cursor.lastrowid

# Вставка данных о приеме
cursor.execute('INSERT INTO appointments (doctor_id, patient_id, appointment_date, consultation_fee) VALUES (?, ?, ?, ?)',
               (doctor_id, patient_id, datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 100.0))

appointment_id = cursor.lastrowid

# Вставка данных о квитанции
cursor.execute('INSERT INTO receipts (appointment_id, amount_paid, receipt_date) VALUES (?, ?, ?)',
               (appointment_id, 100.0, datetime.now().strftime('%Y-%m-%d %H:%M:%S')))

# Расчет зарплаты
cursor.execute('SELECT consultation_fee, fee_percentage FROM doctors WHERE id = ?', (doctor_id,))
consultation_fee, fee_percentage = cursor.fetchone()
gross_salary = consultation_fee * fee_percentage
income_tax = gross_salary * 0.13
net_salary = gross_salary - income_tax

# Вставка данных о зарплате
cursor.execute('INSERT INTO salaries (doctor_id, appointment_id, gross_salary, income_tax, net_salary) VALUES (?, ?, ?, ?, ?)',
               (doctor_id, appointment_id, gross_salary, income_tax, net_salary))

# Сохранение изменений
conn.commit()

# Выборка и вывод данных о зарплате
cursor.execute('SELECT * FROM salaries')
salaries = cursor.fetchall()

for salary in salaries:
    print(f"Doctor ID: {salary[1]}, Gross Salary: {salary[3]}, Income Tax: {salary[4]}, Net Salary: {salary[5]}")

# Закрытие соединения с базой данных
conn.close()


Doctor ID: 1, Gross Salary: 60.0, Income Tax: 7.800000000000001, Net Salary: 52.2


## Задача 3 - Оптовый магазин

Расчет комиссионного вознаграждения продавцов и прибыли от продажи товаров.

In [ ]:
import sqlite3

In [ ]:
from datetime import datetime

In [5]:
# Создаем новое подключение к базе данных SQLite в памяти
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Создаем таблицы для товаров, продавцов и продаж
cursor.execute('''
CREATE TABLE products (
    id INTEGER PRIMARY KEY,
    name TEXT,
    purchase_price REAL,
    sale_price REAL
)''')

cursor.execute('''
CREATE TABLE sellers (
    id INTEGER PRIMARY KEY,
    name TEXT,
    commission_percentage REAL
)''')

cursor.execute('''
CREATE TABLE sales (
    id INTEGER PRIMARY KEY,
    product_id INTEGER,
    seller_id INTEGER,
    quantity INTEGER,
    sale_date TEXT,
    FOREIGN KEY(product_id) REFERENCES products(id),
    FOREIGN KEY(seller_id) REFERENCES sellers(id)
)''')

# Вставка данных о продуктах и продавцах
cursor.execute('INSERT INTO products (name, purchase_price, sale_price) VALUES (?, ?, ?)', 
               ('Product A', 50.0, 70.0))
cursor.execute('INSERT INTO sellers (name, commission_percentage) VALUES (?, ?)', 
               ('Seller 1', 0.05))

# Получаем ID только что вставленных записей
product_id = cursor.lastrowid
seller_id = cursor.lastrowid

# Вставка данных о продаже
cursor.execute('INSERT INTO sales (product_id, seller_id, quantity, sale_date) VALUES (?, ?, ?, ?)',
               (product_id, seller_id, 10, datetime.now().strftime('%Y-%m-%d %H:%M:%S')))

# Сохранение изменений
conn.commit()

# Расчет комиссионного вознаграждения
cursor.execute('''
SELECT s.sale_price, sa.quantity, sl.commission_percentage
FROM sales sa
JOIN products s ON sa.product_id = s.id
JOIN sellers sl ON sa.seller_id = sl.id
''')

sale_price, quantity_sold, commission_percentage = cursor.fetchone()
commission_earned = sale_price * quantity_sold * commission_percentage

# Расчет прибыли от продажи товара
profit = (sale_price - cursor.execute('SELECT purchase_price FROM products WHERE id = ?', (product_id,)).fetchone()[0]) * quantity_sold

# Вывод результатов расчетов
print(f"Commission Earned by Seller: {commission_earned}")
print(f"Profit from Sales: {profit}")

# Закрытие соединения с базой данных
conn.close()


Commission Earned by Seller: 35.0
Profit from Sales: 200.0


## Задача 4 - Сделки с недвижимостью

Расчет комиссионного вознаграждения риэлтора при продаже квартиры.

In [ ]:
import sqlite3

In [ ]:
from datetime import datetime

In [6]:
# Создаем новое подключение к базе данных SQLite в памяти
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Создаем таблицы для квартир, риэлторов и сделок
cursor.execute('''
CREATE TABLE apartments (
    id INTEGER PRIMARY KEY,
    address TEXT,
    price REAL
)''')

cursor.execute('''
CREATE TABLE realtors (
    id INTEGER PRIMARY KEY,
    name TEXT,
    commission_rate REAL
)''')

cursor.execute('''
CREATE TABLE transactions (
    id INTEGER PRIMARY KEY,
    apartment_id INTEGER,
    realtor_id INTEGER,
    transaction_date TEXT,
    FOREIGN KEY(apartment_id) REFERENCES apartments(id),
    FOREIGN KEY(realtor_id) REFERENCES realtors(id)
)''')

# Вставка данных о квартирах и риэлторах
cursor.execute('INSERT INTO apartments (address, price) VALUES (?, ?)', 
               ('123 Main St', 300000.0))
cursor.execute('INSERT INTO realtors (name, commission_rate) VALUES (?, ?)', 
               ('Realtor John', 0.03))

# Получаем ID только что вставленных записей
apartment_id = cursor.lastrowid
realtor_id = cursor.lastrowid

# Вставка данных о сделке
cursor.execute('INSERT INTO transactions (apartment_id, realtor_id, transaction_date) VALUES (?, ?, ?)',
               (apartment_id, realtor_id, datetime.now().strftime('%Y-%m-%d %H:%M:%S')))

# Сохранение изменений
conn.commit()

# Расчет комиссионного вознаграждения риэлтора
cursor.execute('''
SELECT ap.price, rl.commission_rate
FROM transactions tr
JOIN apartments ap ON tr.apartment_id = ap.id
JOIN realtors rl ON tr.realtor_id = rl.id
''')

price, commission_rate = cursor.fetchone()
commission_earned = price * commission_rate

# Вывод результатов расчета комиссии
print(f"Commission Earned by Realtor: {commission_earned}")

# Закрытие соединения с базой данных
conn.close()


Commission Earned by Realtor: 9000.0


## Задача 5 - Досуговый центр

Создание базы данных для управления деятельностью досугового центра.

In [ ]:
import sqlite3

In [ ]:
from datetime import datetime

In [7]:
# Создаем новое подключение к базе данных SQLite в памяти
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Создаем таблицы
cursor.execute('''
CREATE TABLE services (
    id INTEGER PRIMARY KEY,
    name TEXT,
    description TEXT,
    location TEXT, -- 'in-center' или 'at-home'
    price REAL
)''')

cursor.execute('''
CREATE TABLE specialists (
    id INTEGER PRIMARY KEY,
    name TEXT,
    specialty TEXT
)''')

cursor.execute('''
CREATE TABLE orders (
    id INTEGER PRIMARY KEY,
    service_id INTEGER,
    client_id INTEGER,
    specialist_id INTEGER,
    date TEXT,
    FOREIGN KEY(service_id) REFERENCES services(id),
    FOREIGN KEY(client_id) REFERENCES clients(id),
    FOREIGN KEY(specialist_id) REFERENCES specialists(id)
)''')

cursor.execute('''
CREATE TABLE clients (
    id INTEGER PRIMARY KEY,
    name TEXT,
    child_name TEXT,
    phone TEXT,
    email TEXT
)''')

# Вставка примерных данных о услугах
services_data = [
    ('Birthday Party', 'Celebration in center with games and fun activities', 'in-center', 200),
    ('Home Animator', 'Animator visits your home to entertain the kids', 'at-home', 150),
    ('Development Class', 'Games that develop motor skills', 'in-center', 100)
]

cursor.executemany('INSERT INTO services (name, description, location, price) VALUES (?, ?, ?, ?)', services_data)

# Вставка данных о специалистах
specialists_data = [
    ('Alice Johnson', 'Child Psychologist'),
    ('Bob Smith', 'Animator'),
    ('Charlie Davis', 'Early Development Specialist')
]

cursor.executemany('INSERT INTO specialists (name, specialty) VALUES (?, ?)', specialists_data)

# Вставка данных о клиентах
clients_data = [
    ('Jane Doe', 'Little Timmy', '555-1234', 'jane.doe@example.com'),
    ('John Doe', 'Tiny Tina', '555-5678', 'john.doe@example.com')
]

cursor.executemany('INSERT INTO clients (name, child_name, phone, email) VALUES (?, ?, ?, ?)', clients_data)

# Сохранение изменений
conn.commit()

# Выводим добавленные данные о услугах для проверки
cursor.execute('SELECT * FROM services')
print("Services:")
for row in cursor.fetchall():
    print(row)

# Выводим добавленные данные о специалистах для проверки
cursor.execute('SELECT * FROM specialists')
print("\nSpecialists:")
for row in cursor.fetchall():
    print(row)

# Выводим добавленные данные о клиентах для проверки
cursor.execute('SELECT * FROM clients')
print("\nClients:")
for row in cursor.fetchall():
    print(row)

# Закрытие соединения с базой данных
conn.close()


Services:
(1, 'Birthday Party', 'Celebration in center with games and fun activities', 'in-center', 200.0)
(2, 'Home Animator', 'Animator visits your home to entertain the kids', 'at-home', 150.0)
(3, 'Development Class', 'Games that develop motor skills', 'in-center', 100.0)

Specialists:
(1, 'Alice Johnson', 'Child Psychologist')
(2, 'Bob Smith', 'Animator')
(3, 'Charlie Davis', 'Early Development Specialist')

Clients:
(1, 'Jane Doe', 'Little Timmy', '555-1234', 'jane.doe@example.com')
(2, 'John Doe', 'Tiny Tina', '555-5678', 'john.doe@example.com')
